# Calculating the USD Liquidity Index with the OpenBB Platform

This popular indicator is made from a simple subtraction of three FRED series that are published every Wednesday, and is often overlayed with risk assets like the S&P 500 Index or Bitcoin.  The OpenBB SDK is well suited for this task, let's take a look to create this index.

The formula is defined as:

```console
WALCL (All Liabilities) – WLRRAL (RRP) – WDTGAL (TGA)
```

To get these data series, we will use the `openbb-fred` data extension and the `economy` module.  First thing is to import the Python interface, and we will also import Pandas to conduct some DataFrame operations.

In [1]:
from openbb import obb
from pandas import DataFrame

There are two `fred` functions in the `openbb-economy` router:

- `obb.economy.fred_search()`
- `obb.economy.fred_series()`

In [2]:
data = obb.economy.fred_series(["WALCL", "WLRRAL", "WDTGAL", "SP500"])

data

OBBject

id: 066c7874-2012-7189-8000-1e79898a8a3c
results: [{'date': datetime.date(2002, 12, 18), 'WALCL': 719542.0, 'WLRRAL': 21905....
provider: fred
warnings: None
chart: None
extra: {'results_metadata': {'WALCL': {'title': 'Assets: Total Assets: Total Assets...

There is metadata from each series in the warnings of the response object.  It can be recovered as a JSON dictionary.

In [3]:
metadata = data.extra["results_metadata"]

display(metadata.keys())
display(metadata["WALCL"].get("title"))
display(metadata["WALCL"].get("units"))

dict_keys(['WALCL', 'WLRRAL', 'WDTGAL', 'SP500'])

'Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level'

'Millions of U.S. Dollars'

## Querying FRED

If we didn't already know the ID for the series, we can search with:

In [4]:
# The first result is the series we are looking for as the starting value.

obb.economy.fred_search("Wednesday Levels").to_df().head(3)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
0,WALCL,Assets: Total Assets: Total Assets (Less Elimi...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:37:22-05:00,94,94,NaN
1,H41RESPPALDKNWW,Assets: Liquidity and Credit Facilities: Loans...,2002-12-18,2024-08-14,Weekly,W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:37:01-05:00,76,76,NaN
2,TREAST,Assets: Securities Held Outright: U.S. Treasur...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:36:54-05:00,71,71,The total face value of U.S. Treasury securiti...


In [5]:
# Adding "Reverse Repo" to the search returns the second series in the equation, as the first result.

obb.economy.fred_search("Wednesday Levels Reverse Repo").to_df().head(3)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,WLRRAL,Liabilities and Capital: Liabilities: Reverse ...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:37:49-05:00,Reverse repurchase agreements are transactions...,63,63
1,WLRRAFOIAL,Liabilities and Capital: Liabilities: Reverse ...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:37:36-05:00,Reverse repurchase agreements are transactions...,40,40
2,WLRRAOL,Liabilities and Capital: Liabilities: Reverse ...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:37:40-05:00,NaN,29,29


In [6]:
# Refining the search for the Treasury General Account, returns the final series in the equation, as the first result.

obb.economy.fred_search("Wednesday Levels Treasury General").to_df().head(3)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,WDTGAL,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:38:33-05:00,This account is the primary operational accoun...,64,64
1,D2WLTGAL,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:38:37-05:00,NaN,60,60
2,WLDLCL,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2024-08-14,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2024-08-15 15:36:57-05:00,"This item is the sum of ""Term deposits held by...",27,27


In [7]:
# Several major equity indices are published to FRED, S&P 500 is one of them.

obb.economy.fred_search("SP500").to_df().head(2)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,SP500,S&P 500,2014-08-22,2024-08-21,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2024-08-21 19:21:03-05:00,The observations for the S&P 500 represent the...,83,83


By looking at the descriptions, we can confirm that all three Federal Reserve series are numbers as `Millions of USD`.  If they were not all equivalent, some adjustments would need to be made before applying the equation.

In [8]:
for id in metadata:
    display(f"{id}: {metadata[id]['units']}")

'WALCL: Millions of U.S. Dollars'

'WLRRAL: Millions of U.S. Dollars'

'WDTGAL: Millions of U.S. Dollars'

'SP500: Index'

Inspecting the time series element shows that the S&P 500 data (as published to FRED) does not extend as far back as the others.  Let's drop the NaN values and start the time series at a common starting point, which is approximately ten years ago.

In [9]:
display(data.to_df().head(4))
display(data.to_df().dropna().head(4))

# We'll create a new DataFrame object with the dropped rows.
liquidity_index = DataFrame(data.to_df().dropna())

,WALCL,WLRRAL,WDTGAL,SP500
date,,,,
2002-12-18,719542.0,21905.0,6595.0,NaN
2002-12-25,732059.0,20396.0,4662.0,NaN
2003-01-01,730994.0,21091.0,4420.0,NaN
2003-01-08,723762.0,18709.0,5490.0,NaN


,WALCL,WLRRAL,WDTGAL,SP500
date,,,,
2014-08-27,4413736.0,282002.0,29547.0,2000.12
2014-09-03,4415587.0,250306.0,21036.0,2000.72
2014-09-10,4421408.0,267602.0,31872.0,1995.69
2014-09-17,4449588.0,252224.0,123965.0,2001.57


Applying the formula will simply be a matter of subtracting the first, three, columns.

In [10]:
liquidity_index["USD Liquidity Index"] = (
    liquidity_index["WALCL"] - liquidity_index["WLRRAL"] - liquidity_index["WDTGAL"]
)

liquidity_index.tail(4)

,WALCL,WLRRAL,WDTGAL,SP500,USD Liquidity Index
date,,,,,
2024-07-24,7205455.0,805967.0,767419.0,5427.13,5632069.0
2024-07-31,7178391.0,813261.0,854001.0,5522.30,5511129.0
2024-08-07,7175256.0,681881.0,785233.0,5199.50,5708142.0
2024-08-14,7177688.0,722198.0,788823.0,5455.21,5666667.0


Now that there are two items to compare, let's draw it!

In [11]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_scatter(
    x=liquidity_index.index,
    y=liquidity_index["USD Liquidity Index"] / 1000,
    name="USD Liquidity Index (Billions)",
    yaxis="y1",
)

fig.add_scatter(
    x=liquidity_index.index,
    y=liquidity_index["SP500"],
    name="S&P 500 Index",
    yaxis="y2",
)

fig.update_layout(
    yaxis=dict(
        title="USD Liquidity Index (Billions)",
        side="left",
        position=0,
        titlefont=dict(size=12),
        showgrid=False,
    ),
    yaxis2=dict(
        title="S&P 500 Index",
        side="right",
        overlaying="y",
        position=1,
        titlefont=dict(size=12),
    ),
    title="USD Liquidity Index vs. S&P 500 Index",
    title_y=0.90,
    title_x=0.5,
    autosize=True,
)

To draw them both on the same y-axis, they will need to be normalized.  There are several methods for normalizing a series, the fourth function in the block below paramaterizes a few of them, making it easy to A/B them.

In [12]:
y_axis = liquidity_index[["USD Liquidity Index", "SP500"]]


def absolute_maximum_scale(series):
    return series / series.abs().max()


def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())


def z_score_standardization(series):
    return (series - series.mean()) / series.std()


methods = {
    "z": z_score_standardization,
    "m": min_max_scaling,
    "a": absolute_maximum_scale,
}


def normalize(data: DataFrame, method: str = "z") -> DataFrame:
    for col in data.columns:
        data.loc[:, col] = methods[f"{method}"](data.loc[:, col])

    return data


normalized = normalize(y_axis, method="m")

normalized.tail(3)

,USD Liquidity Index,SP500
date,,
2024-07-31,0.643578,0.970490
2024-08-07,0.698835,0.885139
2024-08-14,0.687202,0.952750


Now they can be easily plotted using the built-in `DataFrame.plot` method.

In [13]:
import pandas as pd

pd.options.plotting.backend = "plotly"
normalized.plot()

If you just want to visualize the results, this is a fast way of doing it. However, titles and other customizatons clean things up.

In [14]:
fig = go.Figure()

fig.add_scatter(
    x=normalized.index, y=normalized["USD Liquidity Index"], name="USD Liquidity Index"
)

fig.add_scatter(x=normalized.index, y=normalized["SP500"], name="S&P 500 Index")

fig.update_layout(
    title="USD Liquidity Index vs. S&P 500 Index (Normalized)",
    title_y=0.90,
    title_x=0.5,
    autosize=True,
)

The combinations are endless and we love seeing your creations, tag us on social media with your custom indexes and indicators.